In [1]:
from sporgboost.forests import *
from sklearn.metrics import roc_auc_score
from get_data import load
from sklearn.metrics import roc_auc_score
from sklearn.datasets import load_iris
from sporgboost.preprocessing import onehot_encode
# X, y = load("data/sparse_parity.parquet")
X, y = load_iris(return_X_y=True)
y = onehot_encode(y)


In [34]:
from numba import njit

@njit(cache=False)
def choice_replacement_weighted(X, y, D):
    idx = weighted_draws_replacement(np.arange(X.shape[0]), D, X.shape[0])
    return X[idx, :], y[idx, :]

@njit(cache=False)
def weighted_draws_replacement(a, p, n):
    out = np.empty(shape=(n), dtype='uint')

    for idx in range(n):
        # https://github.com/numba/numba/issues/2539
        out[idx] = a[np.searchsorted(np.cumsum(p), np.random.random(), side="right")]
    return out

choice_replacement_weighted(X, y, 1)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<function cumsum at 0x0000024E0D4025E0>) found for signature:
 
 >>> cumsum(int64)
 
There are 2 candidate implementations:
[1m    - Of which 2 did not match due to:
    Overload of function 'cumsum': File: numba\core\typing\npydecl.py: Line 348.
      With argument(s): '(int64)':[0m
[1m     No match.[0m
[0m
[0m[1mDuring: resolving callee type: Function(<function cumsum at 0x0000024E0D4025E0>)[0m
[0m[1mDuring: typing of call at C:\Users\Ed\AppData\Local\Temp\ipykernel_6560\1267575644.py (15)
[0m
[1m
File "..\..\..\AppData\Local\Temp\ipykernel_6560\1267575644.py", line 15:[0m
[1m<source missing, REPL/exec in use?>[0m

[0m[1mDuring: resolving callee type: type(CPUDispatcher(<function weighted_draws_replacement at 0x0000024E170FE280>))[0m
[0m[1mDuring: typing of call at C:\U

In [22]:
np.random.choice?

Docstring:
choice(a, size=None, replace=True, p=None)

Generates a random sample from a given 1-D array

.. versionadded:: 1.7.0

.. note::
    New code should use the ``choice`` method of a ``default_rng()``
    instance instead; please see the :ref:`random-quick-start`.

Parameters
----------
a : 1-D array-like or int
    If an ndarray, a random sample is generated from its elements.
    If an int, the random sample is generated as if it were ``np.arange(a)``
size : int or tuple of ints, optional
    Output shape.  If the given shape is, e.g., ``(m, n, k)``, then
    ``m * n * k`` samples are drawn.  Default is None, in which case a
    single value is returned.
replace : boolean, optional
    Whether the sample is with or without replacement. Default is True,
    meaning that a value of ``a`` can be selected multiple times.
p : 1-D array-like, optional
    The probabilities associated with each entry in a.
    If not given, the sample assumes a uniform distribution over all
    entr

In [2]:
%%time
ada = AdaBoost()
ada.fit(X, y)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1m- Resolution failure for literal arguments:
[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<built-in method choice of numpy.random.mtrand.RandomState object at 0x0000024E0D795440>) found for signature:

 >>> choice(array(int64, 1d, C), size=int64, replace=Literal[bool](True), p=array(float64, 1d, C))

There are 2 candidate implementations:
[1m   - Of which 2 did not match due to:
   Overload in function 'choice': File: numba\cpython\randomimpl.py: Line 1346.
     With argument(s): '(array(int64, 1d, C), size=int64, replace=bool, p=array(float64, 1d, C))':[0m
[1m    Rejected as the implementation raised a specific error:
      TypeError: choice() got an unexpected keyword argument 'p'[0m
  raised from C:\Users\Ed\anaconda3\envs\proj\lib\site-packages\numba\core\typing\templates.py:722
[0m
[0m[1mDuring: resolving callee type: Fu

In [2]:
from sporgboost.trees import RotationalDecisionTree
rot = RotationalDecisionTree(K=2)
rot.fit(X, y)

In [3]:
from sporgboost.forests import *
rf = RandomForest()
rf.fit(X, y)

In [9]:
sporf = SPORF(d=2, s=3)
sporf.fit(X, y)

In [5]:
rrf = RotationalForest(K=2)
rrf.fit(X, y)

In [19]:
# models = {
#     # 'rf' : RandomForest(),
#     # 'ab' : AdaBoost(),
#     'sporf' : SPORF(d=2, s=3),
#     'sporgboost' : SPORGBoost(d=2, s=3, max_depth=2),
#     # 'rrf' : RotationalRandomForest(K=2),
#     # 'rotboost' : RotBoost(K=2)
# }

# for k, m in models.items():
#     print(f"Fitting {k}")
#     m.fit(X_train, y_train)


Fitting sporf
Fitting sporgboost


In [6]:
roc_auc_score(y, rrf.predict(X), multi_class='ovo')

0.6357403228749623

In [10]:
roc_auc_score(y, sporf.predict_proba(X), multi_class='ovo')

0.976297372398488

In [8]:
roc_auc_score(y, rf.predict_proba(X), multi_class='ovo')

0.5819648490844376

In [24]:
from sklearn.metrics import accuracy_score
# accuracy_score(y_test, rf.predict(X_test))

In [26]:
# Get AUC scores
auc = {key : accuracy_score(y_test, model.predict(X_test)) for key, model in models.items() if key in {'sporf','sporgboost'}}
auc

{'sporf': 0.7630681818181818, 'sporgboost': 0.6098863636363636}